<a href="https://colab.research.google.com/github/sunny0103/DeepLearning_projects/blob/main/%5BNLP%5Daihub_translation/aihub_ko_eng_translate_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [2]:
!pip install  transformers[sentencepiece] accelerate  sacremoses evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 69.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.2 MB/s eta 0:0

In [3]:
import pandas as pd
import numpy as np
import os
import random
from tqdm import tqdm

from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
    )

from datasets import (
    load_dataset,
    DatasetDict,
    load_metric
    )

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [4]:
def seed_everything(seed):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)

seed_everything(42)

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:32"

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
cd "/content/drive/MyDrive/Data/aihub_kor_eng_translate"

/content/drive/MyDrive/Data/aihub_kor_eng_translate


In [7]:
batch_name = f'helsinki_nlp_ko_en_{random.randrange(99999999)}'
batch_name

'helsinki_nlp_ko_en_85822412'

In [8]:
raw_dataset = load_dataset("csv", data_files='total.csv', split='train')
# extract only 20% of the data to check the process and error
train_validtest = raw_dataset.train_test_split(.005)
valid_test = train_validtest['test'].train_test_split(test_size=0.2)

split_datasets = DatasetDict({
    'train': valid_test['train'],
    'valid': valid_test['test'],
})
split_datasets
#############################################################
## the following is considering all the data
# train_validtest = raw_dataset.train_test_split(.2)
# valid_test = train_validtest['test'].train_test_split(test_size=0.5)

# split_datasets = DatasetDict({
#     'train': train_validtest['train'],
#     'valid': valid_test['train'],
#     'test': valid_test['test']
# })
# split_datasets

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-6521daa880b28ffc/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


DatasetDict({
    train: Dataset({
        features: ['index', '원문', '번역문'],
        num_rows: 6410
    })
    valid: Dataset({
        features: ['index', '원문', '번역문'],
        num_rows: 1603
    })
})

In [9]:
print("원문: ", split_datasets['train'][0]['원문'])
print("번역문: ", split_datasets['train'][0]['번역문'])

원문:  머지 않아 탈모 환자가 면역억제제 복용 없이 다른 사람의 건강한 모발을 이식받을 수 있는 길이 열릴 것으로 보인다.
번역문:  Soon, there will be a way for alopecia patients to get the hair implant without taking immunosuppressants.


In [10]:
MODEL_NAME = "Helsinki-NLP/opus-mt-ko-en"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [11]:
max_input_length = 32
max_target_length = 32

def preprocess_function(examples):
  inputs = [ex for ex in examples['원문']]
  targets = [ex for ex in examples['번역문']]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

  # tokenizer setup for target data
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(targets, max_length= max_target_length, truncation=True)

  model_inputs['labels'] = labels['input_ids']
  return model_inputs

In [12]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names)

Map:   0%|          | 0/6410 [00:00<?, ? examples/s]

Map:   0%|          | 0/1603 [00:00<?, ? examples/s]

In [13]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

In [14]:
# dymanic batching
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [15]:
# batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
# batch

In [16]:
metric = load_metric("sacrebleu")

In [17]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # -100은 건너뛴다.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # 단순 후처리
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [18]:
BATCH_SIZE = 16
import torch,  gc
gc.collect()
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

args = Seq2SeqTrainingArguments(
    output_dir = batch_name,
    evaluation_strategy="epoch",
    learning_rate = 5e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=3,
    weight_decay=0.1,
    gradient_accumulation_steps = 8,
    predict_with_generate=True,
    fp16=True, # 고속화 loose한 정확도
    gradient_checkpointing=True # 메모리 절약 대신 느려짐
)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)



In [19]:
trainer.evaluate(max_length=max_target_length)

{'eval_loss': 2.867750406265259,
 'eval_bleu': 14.661331711204703,
 'eval_runtime': 84.2909,
 'eval_samples_per_second': 19.017,
 'eval_steps_per_second': 1.198}

In [20]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu
0,No log,2.371425,18.310989
1,No log,2.301216,19.427510
2,No log,2.285350,19.579577


TrainOutput(global_step=150, training_loss=2.410210978190104, metrics={'train_runtime': 427.6704, 'train_samples_per_second': 44.965, 'train_steps_per_second': 0.351, 'total_flos': 162610456559616.0, 'train_loss': 2.410210978190104, 'epoch': 2.99})

In [21]:
trainer.evaluate(max_length=max_target_length)

{'eval_loss': 2.2853500843048096,
 'eval_bleu': 19.673665152409267,
 'eval_runtime': 83.3087,
 'eval_samples_per_second': 19.242,
 'eval_steps_per_second': 1.212,
 'epoch': 2.99}